In [22]:
import geopandas as gpd
import pandas as pd


In [23]:
path_pref = "/Users/joeyshoyer/Downloads/"

In [24]:
# Load the GeoJSON files
variance_gdf = gpd.read_file(path_pref + "182_Midday_variance.geojson")
midday_gdf = gpd.read_file(path_pref + "182_Midday_speeds.geojson")
pm_peak_gdf = gpd.read_file(path_pref + "182_PM_Peak_speeds.geojson")
am_peak_gdf = gpd.read_file(path_pref + "182_AM_Peak_speeds.geojson")


In [25]:
# Remove rows with missing or None values in stop_id or route_id for each GeoDataFrame
variance_gdf = variance_gdf.dropna(subset=['stop_id', 'route_id'])
midday_gdf = midday_gdf.dropna(subset=['stop_id', 'route_id'])
pm_peak_gdf = pm_peak_gdf.dropna(subset=['stop_id', 'route_id'])
am_peak_gdf = am_peak_gdf.dropna(subset=['stop_id', 'route_id'])


In [26]:
# Create the new column in each GeoDataFrame
variance_gdf['stop_route_id'] = variance_gdf['stop_id'].astype(str) + '_' + variance_gdf['route_id'].astype(str)
midday_gdf['stop_route_id'] = midday_gdf['stop_id'].astype(str) + '_' + midday_gdf['route_id'].astype(str)
pm_peak_gdf['stop_route_id'] = pm_peak_gdf['stop_id'].astype(str) + '_' + pm_peak_gdf['route_id'].astype(str)
am_peak_gdf['stop_route_id'] = am_peak_gdf['stop_id'].astype(str) + '_' + am_peak_gdf['route_id'].astype(str)


In [27]:
variance_gdf = variance_gdf.drop(columns=["geometry", 'id', 'shape_id', 'stop_sequence', 'fast_slow_ratio', 'trips_per_hour', 'miles_from_last', 'route_short_name', 'route_id', 'stop_name', 'stop_id'])
midday_gdf = midday_gdf.drop(columns=['id', 'shape_id', 'stop_sequence', 'direction_id', 'fast_slow_ratio', 'trips_per_hour', 'time_formatted', 'organization_name', 'p20_mph', 'p80_mph', 'stop_id'])
pm_peak_gdf = pm_peak_gdf.drop(columns=["geometry", 'id', 'shape_id', 'stop_sequence', 'direction_id', 'fast_slow_ratio', 'trips_per_hour', 'miles_from_last', 'time_formatted', 'organization_name', 'route_short_name', 'route_id', 'stop_name', 'p20_mph', 'p80_mph', 'stop_id'])
am_peak_gdf = am_peak_gdf.drop(columns=["geometry", 'id', 'shape_id', 'stop_sequence', 'direction_id', 'fast_slow_ratio', 'trips_per_hour', 'miles_from_last', 'time_formatted', 'organization_name', 'route_short_name', 'route_id', 'stop_name', 'p20_mph', 'p80_mph', 'stop_id'])

In [28]:
# Check for duplicates in pm_peak_gdf
midday_gdf.drop_duplicates(subset=['stop_route_id'], keep='first', inplace=True)
pm_peak_gdf.drop_duplicates(subset=['stop_route_id'], keep='first', inplace=True)
am_peak_gdf.drop_duplicates(subset=['stop_route_id'], keep='first', inplace=True)
variance_gdf.drop_duplicates(subset=['stop_route_id'], keep='first', inplace=True)

In [29]:
midday_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 13710 entries, 0 to 19166
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   stop_name         13710 non-null  object  
 1   route_id          13710 non-null  object  
 2   route_short_name  13521 non-null  object  
 3   p50_mph           13710 non-null  float64 
 4   miles_from_last   13710 non-null  float64 
 5   geometry          13710 non-null  geometry
 6   stop_route_id     13710 non-null  object  
dtypes: float64(2), geometry(1), object(4)
memory usage: 856.9+ KB


In [30]:
pm_peak_gdf.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 13916 entries, 0 to 23060
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   p50_mph        13916 non-null  float64
 1   stop_route_id  13916 non-null  object 
dtypes: float64(1), object(1)
memory usage: 326.2+ KB


In [31]:
am_peak_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 13865 entries, 0 to 22776
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   p50_mph        13865 non-null  float64
 1   stop_route_id  13865 non-null  object 
dtypes: float64(1), object(1)
memory usage: 325.0+ KB


In [32]:
# Merge on stop_name
merged_gdf = midday_gdf.merge(pm_peak_gdf, on="stop_route_id", how="inner", suffixes=('_midday', '_pm'))


In [33]:
merged_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13677 entries, 0 to 13676
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   stop_name         13677 non-null  object  
 1   route_id          13677 non-null  object  
 2   route_short_name  13488 non-null  object  
 3   p50_mph_midday    13677 non-null  float64 
 4   miles_from_last   13677 non-null  float64 
 5   geometry          13677 non-null  geometry
 6   stop_route_id     13677 non-null  object  
 7   p50_mph_pm        13677 non-null  float64 
dtypes: float64(3), geometry(1), object(4)
memory usage: 854.9+ KB


In [34]:
merged_gdf = merged_gdf.merge(am_peak_gdf, on="stop_route_id", how="inner", suffixes=('', '_am'))


In [35]:
#merged_gdf = merged_gdf.merge(variance_gdf, on="stop_route_id", how="inner", suffixes=('', '_var'))


In [36]:
merged_gdf = merged_gdf.rename(columns={'p50_mph': 'p50_mph_am'})

In [37]:
merged_gdf.head()

,stop_name,route_id,route_short_name,p50_mph_midday,miles_from_last,geometry,stop_route_id,p50_mph_pm,p50_mph_am
0,Victory / Canoga,169-13172,169,8.7,0.3,"POLYGON ((-118.59655 34.19092, -118.59646 34.1...",7120_169-13172,7.3,10.3
1,Victory / Owensmouth,169-13172,169,8.7,0.2,"POLYGON ((-118.60154 34.18788, -118.60175 34.1...",15495_169-13172,8.8,8.7
2,Topanga Canyon / Victory,169-13172,169,7.0,0.3,"POLYGON ((-118.60604 34.18699, -118.60604 34.1...",6598_169-13172,6.6,6.7
3,Topanga Canyon / Erwin,169-13172,169,9.7,0.3,"POLYGON ((-118.60637 34.18667, -118.60621 34.1...",14946_169-13172,9.7,18.2
4,Topanga Canyon / Oxnard,169-13172,169,18.5,0.3,"POLYGON ((-118.60587 34.17989, -118.60548 34.1...",4190_169-13172,13.1,15.0


In [38]:
merged_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13590 entries, 0 to 13589
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   stop_name         13590 non-null  object  
 1   route_id          13590 non-null  object  
 2   route_short_name  13401 non-null  object  
 3   p50_mph_midday    13590 non-null  float64 
 4   miles_from_last   13590 non-null  float64 
 5   geometry          13590 non-null  geometry
 6   stop_route_id     13590 non-null  object  
 7   p50_mph_pm        13590 non-null  float64 
 8   p50_mph_am        13590 non-null  float64 
dtypes: float64(4), geometry(1), object(4)
memory usage: 955.7+ KB


In [39]:
# Calculate the mean of each p50_mph column
mean_midday = merged_gdf['p50_mph_midday'].mean()
mean_pm = merged_gdf['p50_mph_pm'].mean()
mean_am = merged_gdf['p50_mph_am'].mean()


print(mean_midday, mean_pm, mean_am)

14.835526122148638 13.850419426048566 15.283245033112582


In [40]:
# Calculate the difference from the average for each p50_mph column
merged_gdf['diff_from_avg_midday'] = mean_midday - merged_gdf['p50_mph_midday']
merged_gdf['diff_from_avg_pm'] = mean_pm - merged_gdf['p50_mph_pm']
merged_gdf['diff_from_avg_am'] = mean_am - merged_gdf['p50_mph_am']

In [41]:
merged_gdf.head()


,stop_name,route_id,route_short_name,p50_mph_midday,miles_from_last,geometry,stop_route_id,p50_mph_pm,p50_mph_am,diff_from_avg_midday,diff_from_avg_pm,diff_from_avg_am
0,Victory / Canoga,169-13172,169,8.7,0.3,"POLYGON ((-118.59655 34.19092, -118.59646 34.1...",7120_169-13172,7.3,10.3,6.135526,6.550419,4.983245
1,Victory / Owensmouth,169-13172,169,8.7,0.2,"POLYGON ((-118.60154 34.18788, -118.60175 34.1...",15495_169-13172,8.8,8.7,6.135526,5.050419,6.583245
2,Topanga Canyon / Victory,169-13172,169,7.0,0.3,"POLYGON ((-118.60604 34.18699, -118.60604 34.1...",6598_169-13172,6.6,6.7,7.835526,7.250419,8.583245
3,Topanga Canyon / Erwin,169-13172,169,9.7,0.3,"POLYGON ((-118.60637 34.18667, -118.60621 34.1...",14946_169-13172,9.7,18.2,5.135526,4.150419,-2.916755
4,Topanga Canyon / Oxnard,169-13172,169,18.5,0.3,"POLYGON ((-118.60587 34.17989, -118.60548 34.1...",4190_169-13172,13.1,15.0,-3.664474,0.750419,0.283245


In [42]:
json_df = pd.read_json("/Users/joeyshoyer/Downloads/ridership.json")
json_df['line_name'] = json_df['line_name'].astype(str)
json_df.head()

,year,month,line_name,est_wkday_ridership,est_sat_ridership,est_sun_ridership
0,2009,1,2,21816.0,13442.0,8924.0
1,2009,1,4,20393.0,17941.0,12765.0
2,2009,1,10,13337.0,7603.0,4961.0
3,2009,1,14,16026.0,10696.0,8161.0
4,2009,1,16,26706.0,19855.0,14266.0


In [43]:
# Step 1: Sort by line_name, year, and month in descending order
json_df_sorted = json_df.sort_values(by=['line_name', 'year', 'month'], ascending=[True, False, False])

# Step 2: Drop duplicates based on line_name, keeping the first (most recent) record
json_df_most_recent = json_df_sorted.drop_duplicates(subset='line_name', keep='first')

# Step 3: Inspect the result
print(json_df_most_recent)

       year  month line_name  est_wkday_ridership  est_sat_ridership  \
25443  2024      3        10               7440.0             3664.0   
25468  2024      3       102               1485.0             1135.0   
25469  2024      3       105              14733.0             8856.0   
25470  2024      3       106               4118.0             1860.0   
25471  2024      3       108              14496.0             7759.0   
...     ...    ...       ...                  ...                ...   
25557  2024      3       910              16579.0             7809.0   
25465  2024      3        92               5877.0             3483.0   
3927   2010     12       920               3141.0                NaN   
25466  2024      3        94               6909.0             4724.0   
25467  2024      3        96                752.0              405.0   

       est_sun_ridership  
25443             3463.0  
25468             1016.0  
25469             8378.0  
25470             1820.0  


In [44]:
json_df_most_recent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209 entries, 25443 to 25467
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 209 non-null    int64  
 1   month                209 non-null    int64  
 2   line_name            209 non-null    object 
 3   est_wkday_ridership  208 non-null    float64
 4   est_sat_ridership    155 non-null    float64
 5   est_sun_ridership    152 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 11.4+ KB


In [45]:
json_df_most_recent.sort_values(by='est_wkday_ridership', ascending=False).head(30)

,year,month,line_name,est_wkday_ridership,est_sat_ridership,est_sun_ridership
25550,2024,3,801,67263.0,46467.0,42263.0
25551,2024,3,802,65976.0,50337.0,47508.0
25553,2024,3,804,46550.0,34465.0,30990.0
24499,2023,6,806,33621.0,23574.0,21022.0
4709,2011,6,26,30089.0,24580.0,15512.0
25501,2024,3,207,25334.0,17353.0,15702.0
25442,2024,3,4,23036.0,17788.0,16057.0
25552,2024,3,803,22413.0,11909.0,12906.0
25498,2024,3,204,22131.0,15590.0,15013.0
25446,2024,3,18,20602.0,14632.0,13078.0


In [46]:
merged_gdf = merged_gdf.merge(json_df_most_recent, left_on='route_short_name', right_on='line_name', how='left')

In [47]:
merged_gdf.head()

,stop_name,route_id,route_short_name,p50_mph_midday,miles_from_last,geometry,stop_route_id,p50_mph_pm,p50_mph_am,diff_from_avg_midday,diff_from_avg_pm,diff_from_avg_am,year,month,line_name,est_wkday_ridership,est_sat_ridership,est_sun_ridership
0,Victory / Canoga,169-13172,169,8.7,0.3,"POLYGON ((-118.59655 34.19092, -118.59646 34.1...",7120_169-13172,7.3,10.3,6.135526,6.550419,4.983245,2024.0,3.0,169,1831.0,970.0,826.0
1,Victory / Owensmouth,169-13172,169,8.7,0.2,"POLYGON ((-118.60154 34.18788, -118.60175 34.1...",15495_169-13172,8.8,8.7,6.135526,5.050419,6.583245,2024.0,3.0,169,1831.0,970.0,826.0
2,Topanga Canyon / Victory,169-13172,169,7.0,0.3,"POLYGON ((-118.60604 34.18699, -118.60604 34.1...",6598_169-13172,6.6,6.7,7.835526,7.250419,8.583245,2024.0,3.0,169,1831.0,970.0,826.0
3,Topanga Canyon / Erwin,169-13172,169,9.7,0.3,"POLYGON ((-118.60637 34.18667, -118.60621 34.1...",14946_169-13172,9.7,18.2,5.135526,4.150419,-2.916755,2024.0,3.0,169,1831.0,970.0,826.0
4,Topanga Canyon / Oxnard,169-13172,169,18.5,0.3,"POLYGON ((-118.60587 34.17989, -118.60548 34.1...",4190_169-13172,13.1,15.0,-3.664474,0.750419,0.283245,2024.0,3.0,169,1831.0,970.0,826.0


In [48]:
merged_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13590 entries, 0 to 13589
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   stop_name             13590 non-null  object  
 1   route_id              13590 non-null  object  
 2   route_short_name      13401 non-null  object  
 3   p50_mph_midday        13590 non-null  float64 
 4   miles_from_last       13590 non-null  float64 
 5   geometry              13590 non-null  geometry
 6   stop_route_id         13590 non-null  object  
 7   p50_mph_pm            13590 non-null  float64 
 8   p50_mph_am            13590 non-null  float64 
 9   diff_from_avg_midday  13590 non-null  float64 
 10  diff_from_avg_pm      13590 non-null  float64 
 11  diff_from_avg_am      13590 non-null  float64 
 12  year                  12650 non-null  float64 
 13  month                 12650 non-null  float64 
 14  line_name             12650 non-null  object  

In [49]:
# Time lost for midday, pm, and am
merged_gdf['time_lost_midday'] = (merged_gdf['miles_from_last'] / merged_gdf['diff_from_avg_midday']) * 60
merged_gdf['time_lost_pm'] = (merged_gdf['miles_from_last'] / merged_gdf['diff_from_avg_pm']) * 60
merged_gdf['time_lost_am'] = (merged_gdf['miles_from_last'] / merged_gdf['diff_from_avg_am']) * 60


In [50]:

merged_gdf['ridership_minutes_lost_midday'] = merged_gdf['time_lost_midday'] * merged_gdf['est_wkday_ridership']
merged_gdf['ridership_minutes_lost_pm'] = merged_gdf['time_lost_pm'] * merged_gdf['est_wkday_ridership']
merged_gdf['ridership_minutes_lost_am'] = merged_gdf['time_lost_am'] * merged_gdf['est_wkday_ridership']


In [51]:
merged_gdf[['stop_name', 'route_short_name', 'ridership_minutes_lost_midday', 'ridership_minutes_lost_pm', 'ridership_minutes_lost_am']]


,stop_name,route_short_name,ridership_minutes_lost_midday,ridership_minutes_lost_pm,ridership_minutes_lost_am
0,Victory / Canoga,169,5.371666e+03,5031.433540,6.613763e+03
1,Victory / Owensmouth,169,3.581111e+03,4350.529757,3.337564e+03
2,Topanga Canyon / Victory,169,4.206227e+03,4545.668059,3.839806e+03
3,Topanga Canyon / Erwin,169,6.417648e+03,7940.884189,-1.129954e+04
4,Topanga Canyon / Oxnard,169,-8.993924e+03,43919.438724,1.163586e+05
...,...,...,...,...,...
13585,Franklin / Bronson,207,-6.545212e+05,357480.074759,-1.207936e+05
13586,Imperial / Crenshaw,207,2.650219e+04,16256.383064,-5.595057e+04
13587,Imperial / Ardath,207,3.766646e+04,29512.936215,-7.012711e+05
13588,Imperial / Van Ness,207,1.812127e+06,-282853.319093,1.586494e+06


In [52]:
# Calculate (passenger-hour wasted) / (kilometer travelled)
merged_gdf['passenger_hour_per_mi_midday'] = (merged_gdf['ridership_minutes_lost_midday'] / 60) / merged_gdf['miles_from_last']
merged_gdf['passenger_hour_per_mi_pm'] = (merged_gdf['ridership_minutes_lost_pm'] / 60) / merged_gdf['miles_from_last']
merged_gdf['passenger_hour_per_mi_am'] = (merged_gdf['ridership_minutes_lost_am'] / 60) / merged_gdf['miles_from_last']


In [53]:
# Display results
print("Top 10 segments with highest (passenger-hour wasted) / (mile travelled) for AM peak:")
print(merged_gdf.sort_values('passenger_hour_per_mi_am', ascending=False)[['stop_route_id', 'route_short_name', 'stop_name', 'passenger_hour_per_mi_am']].head(10))

Top 10 segments with highest (passenger-hour wasted) / (mile travelled) for AM peak:
         stop_route_id route_short_name                stop_name  \
12961  25500_207-13172              207        Western / Slauson   
13288   140928_4-13172                4  Santa Monica / La Jolla   
13280     6024_4-13172                4   Santa Monica / La Peer   
11336  14018_204-13172              204       Vermont / Leighton   
9837     14807_2-13172                2            Sunset / Vine   
9846     16330_2-13172                2         Sunset / Gardner   
12726    5372_70-13172               70             Olive / 14th   
12603   10708_70-13172               70           Garvey / Delta   
12796    1464_70-13172               70         Garvey / Potrero   
10351   12703_60-13172               60           Pacific / 46th   

       passenger_hour_per_mi_am  
12961             304330.469372  
13288             276725.218775  
13280             276725.218775  
11336             265853.69928

In [54]:
print("\nTop 10 segments with highest (passenger-hour wasted) / (mile travelled) for PM peak:")
print(merged_gdf.sort_values('passenger_hour_per_mi_pm', ascending=False)[['stop_route_id', 'route_short_name', 'stop_name', 'passenger_hour_per_mi_pm']].head(10))


Top 10 segments with highest (passenger-hour wasted) / (mile travelled) for PM peak:
         stop_route_id route_short_name                   stop_name  \
13321     8029_4-13172                4             Sunset / Benton   
12706   15652_16-13172               16                3rd / Rimpau   
13226    7280_16-13172               16              3rd / Highland   
9791      5150_2-13172                2               Hoover / 28th   
10506     8055_2-13172                2            Sunset / Stanley   
13018   15279_33-13172               33             Venice / Hauser   
13045   14012_33-13172               33            Venice / Lincoln   
11083   1688_105-13172              105           Vernon / McKinley   
11059  14448_105-13172              105  Santa Rosalia / Buckingham   
10738   1998_111-13172              111         Florence / Vinevale   

       passenger_hour_per_mi_pm  
13321             456887.390543  
12706             405121.628722  
13226             405121.62872

In [55]:
print("\nTop 10 segments with highest (passenger-hour wasted) / (mile travelled) for Midday:")
print(merged_gdf.sort_values('passenger_hour_per_mi_midday', ascending=False)[['stop_route_id', 'route_short_name', 'stop_name', 'passenger_hour_per_mi_midday']].head(10))


Top 10 segments with highest (passenger-hour wasted) / (mile travelled) for Midday:
         stop_route_id route_short_name              stop_name  \
12919  16672_720-13172              720     Wilshire / Bonsall   
9876     10988_2-13172                2        Hilgard / Wyton   
10534    17306_2-13172                2       Sunset / Parkman   
9840     16322_2-13172                2      Sunset / Cherokee   
13062    5630_33-13172               33    Venice / Washington   
12174  10520_111-13172              111  Florence / El Selinda   
10751   1968_111-13172              111      Florence / Downey   
10750   1985_111-13172              111   Florence / Paramount   
9547    3167_251-13172              251      Marengo / Mission   
12392    850_210-13172              210       Crenshaw / 147th   

       passenger_hour_per_mi_midday  
12919                 560235.646230  
9876                  534592.543496  
10534                 534592.543496  
9840                  534592.543496 

In [56]:
# Aggregate results by route
route_aggregated = merged_gdf.groupby('route_short_name').agg({
    'passenger_hour_per_mi_am': 'mean',
    'passenger_hour_per_mi_pm': 'mean',
    'passenger_hour_per_mi_midday': 'mean',
    'est_wkday_ridership': 'first'  # Assuming ridership is the same for all segments of a route
}).reset_index()

In [57]:
print("\nTop 10 routes with highest average (passenger-hour wasted) / (mile travelled) for AM peak:")
print(route_aggregated.sort_values('passenger_hour_per_mi_am', ascending=False)[['route_short_name', 'passenger_hour_per_mi_am', 'est_wkday_ridership']].head(10))



Top 10 routes with highest average (passenger-hour wasted) / (mile travelled) for AM peak:
   route_short_name  passenger_hour_per_mi_am  est_wkday_ridership
38              207               8190.770429              25334.0
71                4               8018.745892              23036.0
95              720               5136.557776              19903.0
35              204               4987.953808              22131.0
94               70               4126.087085              15816.0
72               40               2972.684621              14421.0
42              217               2436.636160               8727.0
30              180               2170.584502               9327.0
67               30               1979.765881               9225.0
98              761               1969.721858               7243.0


In [58]:
print("\nTop 10 routes with highest average (passenger-hour wasted) / (mile travelled) for PM peak:")
print(route_aggregated.sort_values('passenger_hour_per_mi_pm', ascending=False)[['route_short_name', 'passenger_hour_per_mi_pm', 'est_wkday_ridership']].head(10))



Top 10 routes with highest average (passenger-hour wasted) / (mile travelled) for PM peak:
   route_short_name  passenger_hour_per_mi_pm  est_wkday_ridership
20               16              12212.254988              20426.0
71                4               5953.700195              23036.0
32                2               4977.558748              18992.0
56              251               4160.350909              14297.0
68               33               4086.765366              16676.0
29               18               3833.283700              20602.0
2               105               3200.876494              14733.0
8               117               3192.313798               8322.0
48              233               3107.714885              12964.0
72               40               3024.552016              14421.0


In [59]:

print("\nTop 10 routes with highest average (passenger-hour wasted) / (mile travelled) for Midday:")
print(route_aggregated.sort_values('passenger_hour_per_mi_midday', ascending=False)[['route_short_name', 'passenger_hour_per_mi_midday', 'est_wkday_ridership']].head(10))


Top 10 routes with highest average (passenger-hour wasted) / (mile travelled) for Midday:
   route_short_name  passenger_hour_per_mi_midday  est_wkday_ridership
95              720                  15434.135364              19903.0
32                2                   9961.595196              18992.0
6               111                   7991.450799              14638.0
79               55                   7319.124733               7375.0
22              162                   6553.967876               9869.0
49              234                   5648.413239               8711.0
56              251                   5238.120182              14297.0
64               28                   4830.211433              10260.0
71                4                   4257.445798              23036.0
72               40                   3845.336686              14421.0


In [60]:
# Calculate total ridership hours wasted for each time period
merged_gdf['ridership_hours_lost_midday'] = merged_gdf['ridership_minutes_lost_midday'] / 60
merged_gdf['ridership_hours_lost_pm'] = merged_gdf['ridership_minutes_lost_pm'] / 60
merged_gdf['ridership_hours_lost_am'] = merged_gdf['ridership_minutes_lost_am'] / 60


In [61]:
# Aggregate results by route
route_aggregated = merged_gdf.groupby('route_short_name').agg({
    'ridership_hours_lost_midday': 'sum',
    'ridership_hours_lost_pm': 'sum',
    'ridership_hours_lost_am': 'sum',
    'est_wkday_ridership': 'first'  # Assuming ridership is the same for all segments of a route
}).reset_index()

In [62]:
# Calculate total ridership hours wasted across all time periods
merged_gdf['total_ridership_hours_lost'] = (
    merged_gdf['ridership_hours_lost_midday'] +
    merged_gdf['ridership_hours_lost_pm'] +
    merged_gdf['ridership_hours_lost_am']
)


In [63]:
# Aggregate results by route
route_aggregated = merged_gdf.groupby('route_short_name').agg({
    'total_ridership_hours_lost': 'sum',
    'est_wkday_ridership': 'first'  # Assuming ridership is the same for all segments of a route
}).reset_index()

# Sort the results by total ridership hours lost
route_aggregated_sorted = route_aggregated.sort_values('total_ridership_hours_lost', ascending=False)


In [64]:
# Display the top 20 routes with highest total ridership hours wasted
print("Top 20 routes with highest total ridership hours wasted:")
print(route_aggregated_sorted[['route_short_name', 'total_ridership_hours_lost', 'est_wkday_ridership']].head(20))


Top 20 routes with highest total ridership hours wasted:
    route_short_name  total_ridership_hours_lost  est_wkday_ridership
95               720               689339.588021              19903.0
71                 4               669130.169124              23036.0
72                40               361868.625406              14421.0
56               251               336756.950348              14297.0
32                 2               289838.356118              18992.0
38               207               238934.899413              25334.0
35               204               211816.188576              22131.0
64                28               173407.412172              10260.0
22               162               170992.335276               9869.0
20                16               165972.481351              20426.0
2                105               147925.873709              14733.0
30               180               137322.735212               9327.0
79                55             

In [65]:
merged_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13590 entries, 0 to 13589
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   stop_name                      13590 non-null  object  
 1   route_id                       13590 non-null  object  
 2   route_short_name               13401 non-null  object  
 3   p50_mph_midday                 13590 non-null  float64 
 4   miles_from_last                13590 non-null  float64 
 5   geometry                       13590 non-null  geometry
 6   stop_route_id                  13590 non-null  object  
 7   p50_mph_pm                     13590 non-null  float64 
 8   p50_mph_am                     13590 non-null  float64 
 9   diff_from_avg_midday           13590 non-null  float64 
 10  diff_from_avg_pm               13590 non-null  float64 
 11  diff_from_avg_am               13590 non-null  float64 
 12  year                    

In [66]:
merged_gdf.head()

,stop_name,route_id,route_short_name,p50_mph_midday,miles_from_last,geometry,stop_route_id,p50_mph_pm,p50_mph_am,diff_from_avg_midday,...,ridership_minutes_lost_midday,ridership_minutes_lost_pm,ridership_minutes_lost_am,passenger_hour_per_mi_midday,passenger_hour_per_mi_pm,passenger_hour_per_mi_am,ridership_hours_lost_midday,ridership_hours_lost_pm,ridership_hours_lost_am,total_ridership_hours_lost
0,Victory / Canoga,169-13172,169,8.7,0.3,"POLYGON ((-118.59655 34.19092, -118.59646 34.1...",7120_169-13172,7.3,10.3,6.135526,...,5371.666479,5031.433540,6613.762675,298.425915,279.524086,367.431260,89.527775,83.857226,110.229378,283.614378
1,Victory / Owensmouth,169-13172,169,8.7,0.2,"POLYGON ((-118.60154 34.18788, -118.60175 34.1...",15495_169-13172,8.8,8.7,6.135526,...,3581.110986,4350.529757,3337.563753,298.425915,362.544146,278.130313,59.685183,72.508829,55.626063,187.820075
2,Topanga Canyon / Victory,169-13172,169,7.0,0.3,"POLYGON ((-118.60604 34.18699, -118.60604 34.1...",6598_169-13172,6.6,6.7,7.835526,...,4206.226804,4545.668059,3839.806492,233.679267,252.537114,213.322583,70.103780,75.761134,63.996775,209.861689
3,Topanga Canyon / Erwin,169-13172,169,9.7,0.3,"POLYGON ((-118.60637 34.18667, -118.60621 34.1...",14946_169-13172,9.7,18.2,5.135526,...,6417.648205,7940.884189,-11299.543628,356.536011,441.160233,-627.752424,106.960803,132.348070,-188.325727,50.983146
4,Topanga Canyon / Oxnard,169-13172,169,18.5,0.3,"POLYGON ((-118.60587 34.17989, -118.60548 34.1...",4190_169-13172,13.1,15.0,-3.664474,...,-8993.924121,43919.438724,116358.615852,-499.662451,2439.968818,6464.367547,-149.898735,731.990645,1939.310264,2521.402174


In [69]:
import json

# Convert GeoDataFrame to a GeoJSON-like Python dictionary
geojson_dict = merged_gdf.__geo_interface__

# Save as GeoJSON
with open("bus_segments.geojson", "w") as f:
    json.dump(geojson_dict, f)

print("Data saved successfully to bus_segments.geojson")

Data saved successfully to bus_segments.geojson
